## Yolo Rock Paper Scissors Notebook

This notebook implements the training process for a finetuned YOLO model using a custom Rock-Paper-Scissors dataset.  
It covers training, evaluation, and live recognition via your notebook camera.

The goal is to build up basic understanding of model training (finetuning) and inference based on a commonly known integration example (rock paper scissors)

### Dependencies and Imports

We start by installing and importing the necessary libraries. This includes:
- `ultralytics` for working with YOLOv8,
- `supervision` and `opencv-python` for image processing and visualization,

In [ ]:
# Install required dependencies
!pip install "ultralytics==8.3.101" "supervision==0.25.1" "roboflow==1.1.54" "opencv-python==4.11.0.86"

In [ ]:
# Import necessary libraries
from ultralytics import YOLO
from IPython.display import Image as IPyImage, display
import numpy as np

### Training new YOLO Model with Rock Paper Scissors Dataset

After training, we visualize key metrics such as:
- the confusion matrix (which shows how well the model differentiates between classes),
- training loss and accuracy plots,
- and prediction examples on validation data.

This allows us to evaluate the model’s performance qualitatively and quantitatively.

In [ ]:

# Load the Yolov11 Original Pretrained Model
#model = YOLO(yolov11_orig_file_path)
model = YOLO('yolov8n.pt')

# Number of epochs to Train
epochs = 1

# Train the model
results = model.train(
    data="https://universe.roboflow.com/ds/HNl4FDwSzM?key=Pa24Xd2c7z",
    epochs=epochs,
    imgsz=640,
    plots=True,
    exist_ok=True,
    device='mps'
)

### Check Model Results

After training, we visualize key metrics such as:

- prediction examples on validation data.

This allows us to evaluate the model’s performance qualitatively and quantitatively.

In [ ]:
# Display training results
display(IPyImage(filename=f'runs/detect/train/val_batch0_pred.jpg', width=600))

### Validate new Trained RPS Model

In [ ]:
model = YOLO("runs/detect/train/weights/best.pt")  # load a trained model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

### Test model with images

In [ ]:
import glob
import shutil
import os

# Match folders using glob pattern
folders = glob.glob('runs/detect/predict*')

for folder in folders:
    if os.path.isdir(folder):
        shutil.rmtree(folder)
        print(f"Deleted: {folder}")

# Load best trained model
try:
    model = YOLO("runs/detect/train/weights/best.pt")
except NameError:
    raise Exception("Model not loaded. Make sure `model` is initialized before running this.")

# Run YOLO prediction on some images
result_fist = model.predict(source="testimages/fist.png", conf=0.1, save=True, verbose=True)
result_paper = model.predict(source="testimages/paper.png", conf=0.1, save=True, verbose=True)
result_scissors = model.predict(source="testimages/scissors.png", conf=0.1, save=True, verbose=True)
result_all = model.predict(source="testimages/all.png", conf=0.1, save=True, verbose=True)

# Display prediction results
predicted_images = glob.glob(f'runs/detect/predict*/*.jpg')

for img in predicted_images:
    display(IPyImage(filename=img, width=600))

### Test model with camera

In [ ]:
import cv2
from IPython.display import display, Image as IPyImage
from PIL import Image
import tempfile
import time
from ultralytics import YOLO


# Check if model is loaded
try:
    model = YOLO("runs/detect/train/weights/best.pt")
except NameError:
    raise Exception("Model not loaded. Make sure `model` is initialized before running this.")

# OpenCV: Open webcam (0 is usually the default camera)
cap = cv2.VideoCapture(0)
# Set resolution to 640x480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)


if not cap.isOpened():
    print("Error: Could not open webcam.")
else:
    print("Starting webcam object detection. Press 'q' to quit.")

    while True:
        # Read frame from the camera
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image")
            break

        # Convert OpenCV's BGR to RGB for YOLO + PIL
        frame_rgb = cv2.cvtColor(frame, cv2.NORMAL_CLONE)

        # Run YOLO prediction on the frame
        results = model.predict(source=frame_rgb, conf=0.1, save=False, verbose=False)

        # Draw results on frame
        annotated_frame = results[0].plot()  # YOLOv8 returns list of results; draw annotations

        # Display annotated frame in real-time using OpenCV
        cv2.imshow('YOLO Live Detection', annotated_frame)

        # Press 'q' to break the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Quitting live detection.")
            break

    # Release the camera and close window
    cap.release()
    cv2.destroyWindow('YOLO Live Detection')
    cv2.waitKey(1)